# マーケットプレイスのデータを使用したSnowflakeでの分析体験
## テーマ: 為替データ分析
このハンズオンでは、Snowflakeの[公開データセット](https://app.snowflake.com/marketplace/listing/GZTSZ290BV255/snowflake-public-data-products-snowflake-public-data-free)を使って為替データの分析方法を学びます（ただし無料版はデータに3ヶ月のラグがありますのでご注意ください）。

**学習内容:**
- Snowflakeの基本的なSQL操作
- 為替データの取得と分析
- データ加工とテーブル作成
- Streamlitを使った可視化
- LLMを使用した分析

![img](https://lh3.googleusercontent.com/pw/AP1GczMB-82X1YDWy5XDmfd3_GqvVuImZtEtexijb27oSWUSglk8LB-qSQY_-7C-Q7iAhNMiewaVccyKDDek6EfR6ZqLgny_LKwksm5_PfM64ynKwmMtPlv_3I6tGwFWhnSQ74OJbSFo3z5jZzL9nRp8pZm2=w960-h540-s-no-gm?authuser=0)

## 事前準備
-  パッケージより`modin`, `matplotlib`, `plotly`をインストール

## MarketplaceからSNOWFLAKE_PUBLIC_DATA_FREEを取得
画面左手のサイドバーより「Marketplace」を選択し、検索ボックスに「Snowflake Public Data Cloud」と入力して下記の画像の画面を開き、右上の取得ボタンをクリック

![img](https://lh3.googleusercontent.com/pw/AP1GczNnsLhzgRDuXN1f9zoOPzGpQGfqd_aNXNO89gAO2dR4zjyn-gfTIqRpsz4wGL4lmnnqoHzXmGFLk8-_Mtbl2-EW1qwXDs2DnOUuOBKm_CkNubpjvhy3DbsBAd3f5mksQz7LqIAThymKesqUZ0PRBKL_=w1501-h882-s-no-gm?authuser=0)


## Part 1: データの確認

まずは、Snowflakeに用意されている為替データを確認しましょう。

### 使用するデータセット
- **データソース**: `SNOWFLAKE_PUBLIC_DATA_FREE.PUBLIC_DATA_FREE.FX_RATES_TIMESERIES`
- **内容**: 複数の通貨ペアの為替レート履歴データ

![img](https://lh3.googleusercontent.com/pw/AP1GczMUr7YO8FSVzpbqiw17FB_LydSg-gUA8BfhgmVCMS-AXvML2b-ZAeOpw4cjQM4A-iIyEKBp4Zkgvq85gxsogA0n8ptch862bG95Dd0nrw-ynpEFL8-Rd8oDlkCFVjEZ7X3ZwhChcIrDf1LlNoLEa8cA=w960-h540-s-no-gm?authuser=0)

In [ ]:
-- 為替データの最初の10件を確認
SELECT *
FROM SNOWFLAKE_PUBLIC_DATA_FREE.PUBLIC_DATA_FREE.FX_RATES_TIMESERIES
LIMIT 10;

**データ項目の説明:**
- `BASE_CURRENCY_ID`: 基軸通貨（例: USD）
- `QUOTE_CURRENCY_ID`: 決済通貨（例: JPY）
- `DATE`: 日付
- `VALUE`: 為替レート
- `VARIABLE_NAME`: 通貨ペア名


### USD/JPYの最新データを確認

USD/JPYの通貨ペアのデータを見てみましょう。

In [ ]:
-- USD/JPYの最新20件のデータを取得
SELECT
    DATE,
    BASE_CURRENCY_ID,
    QUOTE_CURRENCY_ID,
    VALUE AS EXCHANGE_RATE,
    VARIABLE_NAME
FROM SNOWFLAKE_PUBLIC_DATA_FREE.PUBLIC_DATA_FREE.FX_RATES_TIMESERIES
WHERE BASE_CURRENCY_ID = 'USD'
    AND QUOTE_CURRENCY_ID = 'JPY'
ORDER BY DATE DESC
LIMIT 20;

### 利用可能な通貨ペアを確認

どのような通貨ペアのデータがあるか確認してみましょう。


In [ ]:
-- 利用可能な通貨ペアを確認（直近100日分）
SELECT DISTINCT
    BASE_CURRENCY_ID,
    QUOTE_CURRENCY_ID,
    BASE_CURRENCY_NAME,
    QUOTE_CURRENCY_NAME,
    COUNT(*) AS DATA_COUNT
FROM SNOWFLAKE_PUBLIC_DATA_FREE.PUBLIC_DATA_FREE.FX_RATES_TIMESERIES
WHERE DATE >= CURRENT_DATE - 100
GROUP BY 1, 2, 3, 4
ORDER BY BASE_CURRENCY_ID, QUOTE_CURRENCY_ID;

## Part 2: データ分析の基礎
SQLを使って為替データを分析してみましょう。


### 基本統計量の計算

USD/JPYの過去1年間の統計情報を取得します。


In [ ]:
-- USD/JPYの過去1年間の基本統計
SELECT
    BASE_CURRENCY_ID || '/' || QUOTE_CURRENCY_ID AS CURRENCY_PAIR,
    COUNT(*) AS TOTAL_DAYS,
    ROUND(AVG(VALUE), 2) AS AVERAGE_RATE,
    ROUND(MIN(VALUE), 2) AS MIN_RATE,
    ROUND(MAX(VALUE), 2) AS MAX_RATE,
    ROUND(MAX(VALUE) - MIN(VALUE), 2) AS RANGE,
    ROUND(STDDEV(VALUE), 2) AS VOLATILITY
FROM SNOWFLAKE_PUBLIC_DATA_FREE.PUBLIC_DATA_FREE.FX_RATES_TIMESERIES
WHERE BASE_CURRENCY_ID = 'USD'
    AND QUOTE_CURRENCY_ID = 'JPY'
    AND DATE >= CURRENT_DATE - 365
GROUP BY 1;

### 月次の為替レート推移

**TIME_SLICE関数**を使って、月次の平均為替レートを計算します。
![img](https://lh3.googleusercontent.com/pw/AP1GczNZfBn8uy_ynf_oLFdat4KGwl1sextzB-QUzH6vBCL-JnMkWRpWOj03AUeCAvYFX_eDemnJ_QtrIq0YHUcqRKGbPJZpn7UBC7WYCDqnhMEc0Cb6ib-Ia7TVPshmfIzrX3xbHS9d_mLPYBZJi232ivba=w1149-h646-s-no-gm?authuser=0)
- TIME_SLICE 関数を使用して、1週間の平均価格、取引日数、平均最安値、平均最高値を抽出するSQLを簡単に作成
- TIME_SLICE 関数は指定した時間単位に基づいて、タイムスタンプを時間、日、週などの一貫した間隔にデータを分割することが可能
- ユースケース: 特定の銘柄の平均価格や取引量の増減などの傾向を特定など

In [ ]:
-- 月次の平均為替レート
SELECT
    TIME_SLICE(DATE, 1, 'MONTH', 'START') AS month,
    ROUND(AVG(VALUE), 2) AS MONTHLY_AVG_RATE,
    ROUND(MIN(VALUE), 2) AS MONTHLY_MIN_RATE,
    ROUND(MAX(VALUE), 2) AS MONTHLY_MAX_RATE,
    COUNT(*) AS TRADING_DAYS
FROM SNOWFLAKE_PUBLIC_DATA_FREE.PUBLIC_DATA_FREE.FX_RATES_TIMESERIES
WHERE BASE_CURRENCY_ID = 'USD'
    AND QUOTE_CURRENCY_ID = 'JPY'
    AND DATE >= '2024-01-01'
GROUP BY TIME_SLICE(DATE, 1, 'MONTH', 'START')
ORDER BY MONTH DESC;

### 日次変動率の計算

**LAG関数**を使って前日比の変動を計算します。


In [ ]:
-- 日次変動率の計算（最新30日分）
SELECT
    DATE,
    VALUE AS CURRENT_RATE,
    LAG(VALUE, 1) OVER (ORDER BY DATE) AS PREVIOUS_RATE,
    ROUND(VALUE - LAG(VALUE, 1) OVER (ORDER BY DATE), 4) AS DAILY_CHANGE,
    ROUND((VALUE - LAG(VALUE, 1) OVER (ORDER BY DATE)) / LAG(VALUE, 1) OVER (ORDER BY DATE) * 100, 2) AS CHANGE_PERCENT
FROM SNOWFLAKE_PUBLIC_DATA_FREE.PUBLIC_DATA_FREE.FX_RATES_TIMESERIES
WHERE BASE_CURRENCY_ID = 'USD'
    AND QUOTE_CURRENCY_ID = 'JPY'
    AND DATE >= CURRENT_DATE - 200
ORDER BY DATE DESC
LIMIT 30;

## Part 3: データ加工とテーブル作成

分析用のテーブルを作成し、移動平均などの指標を追加します。
![img](https://lh3.googleusercontent.com/pw/AP1GczPgbjDk9s2RF8tVmEN62wy0rgfU5qXf3zmwURB_LdxzjD1qAcVzmcxclSHOtAjX3dmupn5cJRcvISLOo6RZfCg00DVVyQNgIC6b504lpyqDhxjdFsUOBpoA85rGth7bmH3pj1EdatlPcJitYihyuArr=w960-h540-s-no-gm?authuser=0)

### 移動平均を含むテーブルの作成

**移動平均線**は、トレンドを把握するための重要な指標です。

- **5日移動平均**: 短期トレンド
- **20日移動平均**: 中期トレンド
- **50日移動平均**: 長期トレンド


In [ ]:
-- ハンズオン用のドル円為替データに移動平均を追加したテーブルを作成
CREATE OR REPLACE TABLE FX_USDJPY_ANALYSIS AS
SELECT
    DATE,
    BASE_CURRENCY_ID,
    QUOTE_CURRENCY_ID,
    VALUE AS EXCHANGE_RATE,
    
    -- 移動平均の計算
    AVG(VALUE) OVER (
        ORDER BY DATE 
        ROWS BETWEEN 4 PRECEDING AND CURRENT ROW
    ) AS MA_5,
    
    AVG(VALUE) OVER (
        ORDER BY DATE 
        ROWS BETWEEN 19 PRECEDING AND CURRENT ROW
    ) AS MA_20,
    
    AVG(VALUE) OVER (
        ORDER BY DATE 
        ROWS BETWEEN 49 PRECEDING AND CURRENT ROW
    ) AS MA_50,
    
    -- 日次リターン
    (VALUE - LAG(VALUE, 1) OVER (ORDER BY DATE)) / LAG(VALUE, 1) OVER (ORDER BY DATE) * 100 AS DAILY_RETURN
    
FROM SNOWFLAKE_PUBLIC_DATA_FREE.PUBLIC_DATA_FREE.FX_RATES_TIMESERIES
WHERE BASE_CURRENCY_ID = 'USD'
    AND QUOTE_CURRENCY_ID = 'JPY'
    AND DATE >= '2023-01-01'
ORDER BY DATE DESC;

### 作成したテーブルを確認


In [ ]:
-- 作成したテーブルの内容を確認
SELECT *
FROM FX_USDJPY_ANALYSIS
ORDER BY DATE DESC
LIMIT 20;

### 現在のトレンドを分析

移動平均線の位置関係から、現在のトレンドを判断します。


In [ ]:
-- トレンド分析
SELECT
    DATE,
    ROUND(EXCHANGE_RATE, 2) AS RATE,
    ROUND(MA_5, 2) AS MA_5,
    ROUND(MA_20, 2) AS MA_20,
    ROUND(MA_50, 2) AS MA_50,
    CASE
        WHEN EXCHANGE_RATE > MA_5 AND MA_5 > MA_20 AND MA_20 > MA_50 THEN '強い上昇トレンド'
        WHEN EXCHANGE_RATE > MA_20 THEN '上昇トレンド'
        WHEN EXCHANGE_RATE < MA_5 AND MA_5 < MA_20 AND MA_20 < MA_50 THEN '強い下降トレンド'
        WHEN EXCHANGE_RATE < MA_20 THEN '下降トレンド'
        ELSE 'レンジ相場'
    END AS TREND_STATUS
FROM FX_USDJPY_ANALYSIS
WHERE DATE >= CURRENT_DATE - 200
ORDER BY DATE DESC
LIMIT 10;

### 参考）Snowflake Pandas APIを使ってデータ加工と可視化を試す
- Snowpark pandas APIはSnowpark Pythonの一部として提供されるライブラリです。既存の pandas コードをほとんど書き換えずに、Snowflake 上で分散処理として実行できるように設計されています。

Snowpark DataFrame APIを選ぶ場合:
- データエンジニアやSQL開発者で、SQLの思考法に慣れている。
- Apache Spark (PySpark) の経験があり、似たようなAPIを使いたい。

Snowflake Pandas APIを選ぶ場合:
- Pandasの経験があり、その知識を活用したい。
- ローカルで開発したPandasのコードを、最小限の変更でSnowflakeのスケーラブルな環境に移行したい。

In [ ]:
# Snowflake Pandas APIを使ったUSD/JPY為替データの基本分析
# ライブラリのインポート
import modin.pandas as pd
import snowflake.snowpark.modin.plugin
from snowflake.snowpark.context import get_active_session
import matplotlib.pyplot as plt

# データ読み込み
session = get_active_session()
pd.session = session 
df = pd.read_snowflake("FX_USDJPY_ANALYSIS")

# データをソート
df = df.sort_values('DATE')
print(f"\n📊 データ件数: {len(df)}件")

# 過去180日間のデータに絞る（可視化のため）
df_recent = df.tail(180)
print(f"📅 直近180日間のデータで分析: {len(df_recent)}件")

# === データ加工 ===
# 日次リターン（前日比変化率）が既に計算されているか確認
if 'DAILY_RETURN' not in df_recent.columns:
    df_recent['DAILY_RETURN'] = ((df_recent['EXCHANGE_RATE'] - df_recent['EXCHANGE_RATE'].shift(1)) 
                                   / df_recent['EXCHANGE_RATE'].shift(1)) * 100

# 基本統計
print(f"\n📈 基本統計情報:")
print(f"  平均レート: ¥{df_recent['EXCHANGE_RATE'].mean():.2f}")
print(f"  最高値: ¥{df_recent['EXCHANGE_RATE'].max():.2f}")
print(f"  最安値: ¥{df_recent['EXCHANGE_RATE'].min():.2f}")
print(f"  平均日次変動率: {df_recent['DAILY_RETURN'].mean():.2f}%")
print(f"  ボラティリティ（標準偏差）: {df_recent['EXCHANGE_RATE'].std():.2f}")

# === 可視化 ===
plt.figure(figsize=(15, 8))

# 1. 為替レートチャート（移動平均付き）
plt.subplot(2, 1, 1)
plt.plot(df_recent['DATE'], df_recent['EXCHANGE_RATE'], 
         label='USD/JPY Rate', color='blue', linewidth=2)
plt.plot(df_recent['DATE'], df_recent['MA_20'], 
         label='20-day MA', color='orange', linestyle='--', linewidth=1.5)
plt.plot(df_recent['DATE'], df_recent['MA_50'], 
         label='50-day MA', color='red', linestyle='--', linewidth=1.5)
plt.title('USD/JPY Exchange Rate Trend', fontsize=14, fontweight='bold')
plt.ylabel('Rate (JPY)', fontsize=12)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)

# 2. 日次変動率チャート
plt.subplot(2, 1, 2)
# 日次変動率に応じて色を設定（正の値は緑、負の値は赤）
colors = ['red' if x < 0 else 'green' for x in df_recent['DAILY_RETURN']]
plt.bar(df_recent['DATE'], df_recent['DAILY_RETURN'], color=colors, alpha=0.6)
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.title('Daily Return (%)', fontsize=14, fontweight='bold')
plt.ylabel('Return (%)', fontsize=12)
plt.xlabel('Date', fontsize=12)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 4: Streamlitで可視化

Streamlitを使って、為替データを可視化するダッシュボードを作成します。

### Streamlitとは？
- PythonだけでWebアプリを作成できるフレームワーク
- HTML/CSSの知識不要
- データサイエンスのプロトタイプ作成に最適

![img](https://lh3.googleusercontent.com/pw/AP1GczNDxLHiIOf4GU6YBm35uIoPXMzzkTeOq0sJcwAhOOOdTrPKDmZmzx1WgWEIAkwM5T6syE7_ZBsS11tslwoxGBBQg3OacwZBndbwi3I6_F-Zv5YW8IEhctbk-fm5EGm4kzKE9nqdB5uHCzb1TVl_9e2y=w960-h540-s-no-gm?authuser=0)

![img](https://lh3.googleusercontent.com/pw/AP1GczMhp7CFMU7LnrNkc6izujuwTnUDfVTkOAtAWF-pBQsOONqjsrsllpe7OVUn66AhcJNtommDtzUnHl0Los8H88rIqBz-n2yajpR9Z4HJzjxXrLb_6f77KRBUI4kQ9mnehPYUXDdA66OwsnYg2wwWuz_R=w1042-h617-s-no-gm?authuser=0)

### Step 1: 基本構造 - タイトルとデータ取得

まずは、タイトルを表示してデータを取得する基本的な構造を作ります。
#### 参考）Snowpark Pythonを使った表現
- Snowpark Python は、Snowflake 上で「データを外に持ち出さずに」Python コードを実行できる開発フレームワーク
- つまりSnowflake 内で完結する大規模データ処理を、Python の書き慣れた文法で実装できる」フレームワーク
- SQL と Python をシームレスに組み合わせたいデータエンジニア／データサイエンティストに最適なソリューション


In [ ]:
# 必要なライブラリをインポート
import streamlit as st
import pandas as pd
from snowflake.snowpark.context import get_active_session
from datetime import date, timedelta

# タイトル設定
st.title("💱 USD/JPY 為替レート分析")
st.markdown("### Snowflake ハンズオン")

# Snowflakeセッション取得
session = get_active_session()

# データ取得関数（キャッシュ機能付き）
@st.cache_data
def get_fx_data(start_date, end_date):
    """為替データを取得する関数"""
    query = f"""
    SELECT * FROM FX_USDJPY_ANALYSIS
    WHERE DATE >= '{start_date}' AND DATE <= '{end_date}'
    ORDER BY DATE ASC
    """
    result = session.sql(query)
    return result.to_pandas()

# デフォルト日付設定（過去6ヶ月）
default_start = date.today() - timedelta(days=180)
default_end = date.today()

df = get_fx_data(default_start, default_end)

# データ確認
st.markdown("#### 📊 取得したデータの確認")
st.write(f"期間: {default_start} ～ {default_end}")
st.write(f"データ件数: {len(df)}件")
st.dataframe(df.head(10))

### Step 2: 日付選択とメトリクス表示

ユーザーが期間を選択できるようにし、重要な指標を表示します。


In [ ]:
# 日付選択UI
st.markdown("### 📅 期間選択")
col1, col2 = st.columns(2)

with col1:
    start_date = st.date_input(
        "開始日",
        value=date.today() - timedelta(days=180)
    )

with col2:
    end_date = st.date_input(
        "終了日",
        value=date.today()
    )

# データ取得
with st.spinner(f"📊 {start_date} から {end_date} のデータを読み込み中..."):
    df = get_fx_data(start_date, end_date)

# データが空の場合の処理
if df.empty:
    st.warning("⚠️ 選択した期間にはデータがありません。期間を変更してください。")
    st.stop()

# メトリクス計算
latest_rate = df['EXCHANGE_RATE'].iloc[-1]
previous_rate = df['EXCHANGE_RATE'].iloc[-2] if len(df) > 1 else latest_rate
rate_change = ((latest_rate / previous_rate) - 1) * 100 if previous_rate != 0 else 0
latest_ma50 = df['MA_50'].iloc[-1] if pd.notna(df['MA_50'].iloc[-1]) else 0

# メトリクス表示
st.markdown("### 📊 主要指標")
col1, col2, col3, col4 = st.columns(4)

with col1:
    st.metric(
        "現在レート",
        f"¥{latest_rate:.2f}",
        f"{rate_change:+.2f}%" if len(df) > 1 else "N/A"
    )

with col2:
    st.metric(
        "期間最高値",
        f"¥{df['EXCHANGE_RATE'].max():.2f}"
    )

with col3:
    st.metric(
        "期間最安値",
        f"¥{df['EXCHANGE_RATE'].min():.2f}"
    )

with col4:
    st.metric(
        "50日移動平均",
        f"¥{latest_ma50:.2f}",
        f"{'レートより上' if latest_rate > latest_ma50 else 'レートより下'}"
    )


### Step 3: 統計情報の追加

In [ ]:
# 統計情報の表示
st.markdown("### 📈 統計情報")
col1, col2 = st.columns(2)

with col1:
    st.markdown("#### 為替レート統計")
    rate_stats = pd.DataFrame({
        '指標': ['現在レート', '平均レート', '最高値', '最安値', '変動幅', 'ボラティリティ'],
        '値': [
            f"¥{df['EXCHANGE_RATE'].iloc[-1]:.2f}",
            f"¥{df['EXCHANGE_RATE'].mean():.2f}",
            f"¥{df['EXCHANGE_RATE'].max():.2f}",
            f"¥{df['EXCHANGE_RATE'].min():.2f}",
            f"¥{df['EXCHANGE_RATE'].max() - df['EXCHANGE_RATE'].min():.2f}",
            f"{df['EXCHANGE_RATE'].std():.2f}"
        ]
    })
    st.dataframe(rate_stats, hide_index=True, use_container_width=True)

with col2:
    st.markdown("#### 移動平均線")
    ma_stats = pd.DataFrame({
        '指標': ['5日移動平均', '20日移動平均', '50日移動平均', '現在の位置'],
        '値': [
            f"¥{df['MA_5'].iloc[-1]:.2f}" if pd.notna(df['MA_5'].iloc[-1]) else 'N/A',
            f"¥{df['MA_20'].iloc[-1]:.2f}" if pd.notna(df['MA_20'].iloc[-1]) else 'N/A',
            f"¥{df['MA_50'].iloc[-1]:.2f}" if pd.notna(df['MA_50'].iloc[-1]) else 'N/A',
            '移動平均線より上' if latest_rate > latest_ma50 else '移動平均線より下'
        ]
    })
    st.dataframe(ma_stats, hide_index=True, use_container_width=True)


### Step 4: グラフの作成

Plotlyを使って、為替レートと移動平均線をグラフ化します。


In [ ]:
import plotly.graph_objects as go

st.markdown("### 📈 為替レート推移グラフ")

# グラフの作成
fig = go.Figure()

# 為替レート
fig.add_trace(
    go.Scatter(
        x=df['DATE'],
        y=df['EXCHANGE_RATE'],
        mode='lines',
        name='為替レート',
        line=dict(color='#1f77b4', width=2),
        hovertemplate='日付: %{x|%Y-%m-%d}<br>レート: ¥%{y:.2f}<extra></extra>'
    )
)

# 5日移動平均
fig.add_trace(
    go.Scatter(
        x=df['DATE'],
        y=df['MA_5'],
        mode='lines',
        name='5日移動平均',
        line=dict(color='#90EE90', width=1),
        hovertemplate='日付: %{x|%Y-%m-%d}<br>5日MA: ¥%{y:.2f}<extra></extra>'
    )
)

# 20日移動平均
fig.add_trace(
    go.Scatter(
        x=df['DATE'],
        y=df['MA_20'],
        mode='lines',
        name='20日移動平均',
        line=dict(color='#FFA500', width=1),
        hovertemplate='日付: %{x|%Y-%m-%d}<br>20日MA: ¥%{y:.2f}<extra></extra>'
    )
)

# 50日移動平均
fig.add_trace(
    go.Scatter(
        x=df['DATE'],
        y=df['MA_50'],
        mode='lines',
        name='50日移動平均',
        line=dict(color='#FF4444', width=1, dash='dash'),
        hovertemplate='日付: %{x|%Y-%m-%d}<br>50日MA: ¥%{y:.2f}<extra></extra>'
    )
)

# レイアウト設定
fig.update_layout(
    title='USD/JPY 為替レートと移動平均線',
    xaxis_title='日付',
    yaxis_title='為替レート (円)',
    height=500,
    hovermode='x unified',
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

# グラフ表示
st.plotly_chart(fig, use_container_width=True)


### Step 5: データテーブルの表示

エキスパンダーを使って、詳細なデータテーブルを表示します。


In [ ]:
# データテーブルの表示
with st.expander("📋 詳細データを表示", expanded=False):
    display_df = df.sort_values('DATE', ascending=False).copy()
    display_df['DATE'] = pd.to_datetime(display_df['DATE']).dt.strftime('%Y-%m-%d')
    
    # 列名を日本語に変更
    display_df = display_df.rename(columns={
        'DATE': '日付',
        'EXCHANGE_RATE': '為替レート',
        'MA_5': '5日移動平均',
        'MA_20': '20日移動平均',
        'MA_50': '50日移動平均',
        'DAILY_RETURN': '日次変動率(%)'
    })
    
    # 数値フォーマット
    st.dataframe(
        display_df[['日付', '為替レート', '5日移動平均', '20日移動平均', '50日移動平均', '日次変動率(%)']].style.format({
            '為替レート': '¥{:.2f}',
            '5日移動平均': '¥{:.2f}',
            '20日移動平均': '¥{:.2f}',
            '50日移動平均': '¥{:.2f}',
            '日次変動率(%)': '{:.2f}%'
        }),
        use_container_width=True,
        height=400
    )


### Step 6: 完成版 - 全要素を組み合わせる

これまでのステップを1つにまとめた完成版のStreamlitアプリです。

In [ ]:
# ============================================
# 完成版: USD/JPY 為替レート分析ダッシュボード
# ============================================

import streamlit as st
import pandas as pd
import plotly.graph_objects as go
from snowflake.snowpark.context import get_active_session
from datetime import date, timedelta

# タイトル
st.title("💱 USD/JPY 為替レート分析ダッシュボード")
st.markdown("### Snowflake ハンズオン")
st.markdown("---")

# Snowflakeセッション取得
session = get_active_session()

# データ取得関数
@st.cache_data
def get_fx_data(start_date, end_date):
    query = f"""
    SELECT * FROM FX_USDJPY_ANALYSIS
    WHERE DATE >= '{start_date}' AND DATE <= '{end_date}'
    ORDER BY DATE ASC
    """
    result = session.sql(query)
    return result.to_pandas()

# 期間選択（メインエリア）
st.subheader("📅 期間選択")
col1, col2 = st.columns(2)

with col1:
    start_date = st.date_input(
        "開始日",
        value=date.today() - timedelta(days=180),
        max_value=date.today()
    )

with col2:
    end_date = st.date_input(
        "終了日",
        value=date.today(),
        max_value=date.today()
    )

# 日付の妥当性チェック
if start_date > end_date:
    st.error("❌ 開始日は終了日より前の日付を選択してください。")
    st.stop()

st.markdown("---")

# データ読み込み
with st.spinner("データを読み込んでいます..."):
    df = get_fx_data(start_date, end_date)

if df.empty:
    st.warning("選択した期間にはデータがありません。期間を変更してください。")
    st.stop()

# メトリクス計算
latest_rate = df['EXCHANGE_RATE'].iloc[-1]
previous_rate = df['EXCHANGE_RATE'].iloc[-2] if len(df) > 1 else latest_rate
rate_change = ((latest_rate / previous_rate) - 1) * 100 if previous_rate != 0 else 0
latest_ma50 = df['MA_50'].iloc[-1] if pd.notna(df['MA_50'].iloc[-1]) else 0
max_rate = df['EXCHANGE_RATE'].max()
min_rate = df['EXCHANGE_RATE'].min()

# メトリクス表示
col1, col2, col3, col4 = st.columns(4)

with col1:
    st.metric(
        "現在レート",
        f"¥{latest_rate:.2f}",
        f"{rate_change:+.2f}%" if len(df) > 1 else "N/A"
    )

with col2:
    st.metric(
        "期間最高値",
        f"¥{max_rate:.2f}"
    )

with col3:
    st.metric(
        "期間最安値",
        f"¥{min_rate:.2f}"
    )

with col4:
    st.metric(
        "50日移動平均",
        f"¥{latest_ma50:.2f}",
        f"{'レートより上' if latest_rate > latest_ma50 else 'レートより下'}"
    )

st.markdown("---")

# グラフ表示
st.subheader("📈 為替レート推移")

fig = go.Figure()

# 為替レート
fig.add_trace(
    go.Scatter(
        x=df['DATE'],
        y=df['EXCHANGE_RATE'],
        mode='lines',
        name='為替レート',
        line=dict(color='#1f77b4', width=2),
        hovertemplate='日付: %{x|%Y-%m-%d}<br>レート: ¥%{y:.2f}<extra></extra>'
    )
)

# 移動平均線
fig.add_trace(
    go.Scatter(
        x=df['DATE'],
        y=df['MA_5'],
        mode='lines',
        name='5日移動平均',
        line=dict(color='#90EE90', width=1)
    )
)

fig.add_trace(
    go.Scatter(
        x=df['DATE'],
        y=df['MA_20'],
        mode='lines',
        name='20日移動平均',
        line=dict(color='#FFA500', width=1)
    )
)

fig.add_trace(
    go.Scatter(
        x=df['DATE'],
        y=df['MA_50'],
        mode='lines',
        name='50日移動平均',
        line=dict(color='#FF4444', width=1, dash='dash')
    )
)

fig.update_layout(
    xaxis_title='日付',
    yaxis_title='為替レート (円)',
    height=500,
    hovermode='x unified',
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

st.plotly_chart(fig, use_container_width=True)

# 統計情報
st.markdown("---")
st.subheader("📊 統計情報")

col1, col2 = st.columns(2)

with col1:
    st.markdown("#### 為替レート統計")
    rate_stats = pd.DataFrame({
        '指標': ['現在レート', '平均レート', '最高値', '最安値', '変動幅', 'ボラティリティ'],
        '値': [
            f"¥{latest_rate:.2f}",
            f"¥{df['EXCHANGE_RATE'].mean():.2f}",
            f"¥{max_rate:.2f}",
            f"¥{min_rate:.2f}",
            f"¥{max_rate - min_rate:.2f}",
            f"{df['EXCHANGE_RATE'].std():.2f}"
        ]
    })
    st.dataframe(rate_stats, hide_index=True, use_container_width=True)

with col2:
    st.markdown("#### 移動平均線")
    ma_stats = pd.DataFrame({
        '指標': ['5日移動平均', '20日移動平均', '50日移動平均', '現在の位置'],
        '値': [
            f"¥{df['MA_5'].iloc[-1]:.2f}" if pd.notna(df['MA_5'].iloc[-1]) else 'N/A',
            f"¥{df['MA_20'].iloc[-1]:.2f}" if pd.notna(df['MA_20'].iloc[-1]) else 'N/A',
            f"¥{latest_ma50:.2f}",
            '移動平均線より上' if latest_rate > latest_ma50 else '移動平均線より下'
        ]
    })
    st.dataframe(ma_stats, hide_index=True, use_container_width=True)

# 詳細データ
with st.expander("📋 詳細データを表示", expanded=False):
    display_df = df.sort_values('DATE', ascending=False).copy()
    display_df['DATE'] = pd.to_datetime(display_df['DATE']).dt.strftime('%Y-%m-%d')
    
    display_df = display_df.rename(columns={
        'DATE': '日付',
        'EXCHANGE_RATE': '為替レート',
        'MA_5': '5日移動平均',
        'MA_20': '20日移動平均',
        'MA_50': '50日移動平均',
        'DAILY_RETURN': '日次変動率(%)'
    })
    
    st.dataframe(
        display_df[['日付', '為替レート', '5日移動平均', '20日移動平均', '50日移動平均', '日次変動率(%)']].style.format({
            '為替レート': '¥{:.2f}',
            '5日移動平均': '¥{:.2f}',
            '20日移動平均': '¥{:.2f}',
            '50日移動平均': '¥{:.2f}',
            '日次変動率(%)': '{:.2f}%'
        }),
        use_container_width=True,
        height=400
    )

# フッター
st.markdown("---")
st.markdown(
    """
    <div style='text-align: center; color: #666;'>
        <p>💱 USD/JPY 為替レート分析ダッシュボード | Powered by Snowflake & Streamlit</p>
        <p style='font-size: 0.8rem;'>Data Source: SNOWFLAKE_PUBLIC_DATA_FREE.PUBLIC_DATA_FREE.FX_RATES_TIMESERIES</p>
    </div>
    """,
    unsafe_allow_html=True
)


## Part 5: AI分析（Snowflake Cortex）

Snowflake CortexのAI機能を使って、為替データをより深く分析します。

### Snowflake Cortex AI機能とは？
- **AI_COMPLETE**: LLMを使った自然言語による分析・洞察の生成
- **AI_FILTER**: データを自然言語の条件でフィルタリング

## ハンズオン3: LLM を使った分析
- AISQL を使った分析を試していく（詳細は後半のハンズオンにて）
![img](https://lh3.googleusercontent.com/pw/AP1GczPEstMLsFYzhztvUUHigYdBALM3BDLmvKUQKWDMmCQKuebzruOhKZpKoZeMH1kcoVBil8SUwscciE4_z4AtPx6UrRPwfLt6nhJUS1Lg-q8arZy_TAW6-ZYLdGplQedh7XQxLIgcfYobF6Lk8jLAcn10=w1149-h646-s-no-gm?authuser=0)

In [ ]:
-- SQL の場合
select AI_COMPLETE ('openai-gpt-4.1', 'Snowflakeの特徴を50字以内で教えてください。');

In [ ]:
-- AI_COMPLETEで分析（4秒ほどで出力）
-- 最近200日間のデータをもとにAIが為替トレンドを分析
WITH recent_data AS (
    SELECT 
        ROUND(AVG(EXCHANGE_RATE), 2) AS avg_rate,
        ROUND(MIN(EXCHANGE_RATE), 2) AS min_rate,
        ROUND(MAX(EXCHANGE_RATE), 2) AS max_rate,
        ROUND(STDDEV(EXCHANGE_RATE), 2) AS volatility,
        ROUND(MAX(EXCHANGE_RATE) - MIN(EXCHANGE_RATE), 2) AS rate_range,
        COUNT(*) AS trading_days
    FROM FX_USDJPY_ANALYSIS
    WHERE DATE >= CURRENT_DATE - 200
)
SELECT 
    AI_COMPLETE(
        'openai-gpt-4.1',
        CONCAT(
            'あなたはウォール街の為替専門のエコノミストです。USD/JPY為替レートの最近30日間のデータを分析してください。',
            '\n\n【データ概要】',
            '\n- 平均レート: ', avg_rate, '円',
            '\n- 最高値: ', max_rate, '円',
            '\n- 最安値: ', min_rate, '円',
            '\n- 変動幅: ', rate_range, '円',
            '\n- ボラティリティ(標準偏差): ', volatility,
            '\n- 取引日数: ', trading_days, '日',
            '\n\n以下の観点から簡潔に分析してください：',
            '\n1. 現在の市場状況（2-3行）',
            '\n2. トレンドの方向性（2-3行）',
            '\n3. 注目すべきポイント（2-3行）'
        )
    ) AS ai_market_analysis
FROM recent_data;

In [ ]:
-- AI_FILTERを使って大きな変動があった日を検出(50秒ほどで出力)
WITH daily_changes AS (
    SELECT 
        DATE,
        EXCHANGE_RATE,
        DAILY_RETURN,
        ABS(DAILY_RETURN) AS abs_return,
        CONCAT(
            '日付: ', DATE,
            ', 為替レート: ', ROUND(EXCHANGE_RATE, 2), '円',
            ', 日次変動率: ', ROUND(DAILY_RETURN, 2), '%'
        ) AS change_description
    FROM FX_USDJPY_ANALYSIS
    WHERE DATE >= CURRENT_DATE - 365
        AND DAILY_RETURN IS NOT NULL
)
SELECT 
    DATE,
    ROUND(EXCHANGE_RATE, 2) AS RATE,
    ROUND(DAILY_RETURN, 2) AS RETURN_PCT,
    change_description
FROM daily_changes
WHERE AI_FILTER(
    CONCAT('この日の為替相場で一般的に変動率は大きいか？（1%以上の変動または平均から大きく乖離）: ', change_description)
)
ORDER BY DATE DESC
LIMIT 10;

In [ ]:
-- Step 1: AI_FILTERで異常変動日を検出
-- Step 2: その日のデータをAI_COMPLETEで分析（1分ほどで出力）
WITH anomaly_days AS (
    SELECT 
        DATE,
        EXCHANGE_RATE,
        DAILY_RETURN,
        MA_20,
        CONCAT(
            '日付: ', DATE, 
            ', レート: ', ROUND(EXCHANGE_RATE, 2), '円',
            ', 日次変動: ', ROUND(DAILY_RETURN, 2), '%',
            ', 20日平均との乖離: ', ROUND(((EXCHANGE_RATE - MA_20) / MA_20 * 100), 2), '%'
        ) AS day_summary
    FROM FX_USDJPY_ANALYSIS
    WHERE DATE >= CURRENT_DATE - 365
        AND DAILY_RETURN IS NOT NULL
        AND MA_20 IS NOT NULL
        AND AI_FILTER(
            CONCAT(
                'この日の為替相場で一般的に変動率は大きいか？（1%以上の変動または平均から大きく乖離）: ',
                '日次変動率: ', ROUND(DAILY_RETURN, 2), '%'
            )
        )
    ORDER BY DATE DESC
    LIMIT 3
)
SELECT 
    DATE AS date,
    ROUND(EXCHANGE_RATE, 2) AS rate,
    ROUND(DAILY_RETURN, 2) AS pct_change,
     AI_COMPLETE(
        'openai-gpt-4.1',
        CONCAT(
            'あなたはウォール街の為替専門のエコノミストです。USD/JPY為替レートで以下の日に為替相場の一般的な市場において異常な変動がありました。',
            '\n\n', day_summary,
            '\n\nこの変動について、考えられる要因と注意点を2-3行で簡潔に説明してください。'
        )
    ) AS analysis_result
FROM anomaly_days;